In [13]:
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
import numpy as np
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image, image_dataset_from_directory
import matplotlib.pyplot as plt

ImportError: cannot import name 'image_dataset_from_directory' from 'keras.preprocessing' (/Users/filipkatulski/opt/anaconda3/lib/python3.7/site-packages/keras/preprocessing/__init__.py)

In [10]:
def preprocess_image(directory):
    image_dataset_from_directory(directory, image_size=(640, 480))
    return img

In [11]:
img1 = preprocess_image('/Users/filipkatulski/Documents/OneDrive/Studia/12 SEMESTR/ML/lab1/iros2021/DataSet_Nao_RAW/DataSet_SEQUENCE_1')

NameError: name 'keras' is not defined

In [4]:
img1

array([[[[ -30.939003,  -46.779   ,  -57.68    ],
         [ -29.939003,  -43.779   ,  -60.68    ],
         [ -30.939003,  -44.779   ,  -61.68    ],
         ...,
         [ -88.939   ,  -91.779   , -100.68    ],
         [ -87.939   ,  -90.779   , -100.68    ],
         [ -94.939   ,  -88.779   , -105.68    ]],

        [[ -25.939003,  -42.779   ,  -58.68    ],
         [ -25.939003,  -39.779   ,  -56.68    ],
         [ -27.939003,  -40.779   ,  -60.68    ],
         ...,
         [ -78.939   ,  -91.779   ,  -98.68    ],
         [ -86.939   ,  -89.779   ,  -98.68    ],
         [ -95.939   ,  -87.779   , -104.68    ]],

        [[ -25.939003,  -40.779   ,  -55.68    ],
         [ -25.939003,  -40.779   ,  -55.68    ],
         [ -25.939003,  -40.779   ,  -55.68    ],
         ...,
         [ -87.939   ,  -88.779   , -104.68    ],
         [ -86.939   ,  -89.779   ,  -98.68    ],
         [ -88.939   ,  -89.779   , -105.68    ]],

        ...,

        [[ -55.939003,  -59.779   ,  -

In [5]:
img1.shape

(1, 224, 224, 3)